In [1]:
import os, json, hashlib, tempfile
from pathlib import Path
import ipywidgets as W
from IPython.display import display, HTML
import papermill as pm
import nbformat
from nbconvert import HTMLExporter

In [2]:
ARTIFACTS = Path("artifacts")

def _cache_dir_for(proj: str, run: str) -> Path:
    d = ARTIFACTS / proj / "runs" / run / "panel" / "embed"
    d.mkdir(parents=True, exist_ok=True)
    return d

def _hash_params(nb_path: str, params: dict) -> str:
    h = hashlib.sha256()
    h.update(nb_path.encode())
    h.update(json.dumps(params, sort_keys=True, ensure_ascii=False).encode("utf-8"))
    return h.hexdigest()[:16]

def execute_notebook_to_html_cached(nb_path: str, params: dict, cache_dir: Path) -> str:
    key = _hash_params(nb_path, params)
    cache_file = cache_dir / f"{Path(nb_path).stem}-{key}.html"
    if cache_file.exists():
        return cache_file.read_text(encoding="utf-8")

    with tempfile.TemporaryDirectory() as td:
        out_ipynb = Path(td) / "executed.ipynb"
        pm.execute_notebook(nb_path, str(out_ipynb), parameters=params, report_mode=True, progress_bar=False)
        nb = nbformat.read(out_ipynb, as_version=4)
        exp = HTMLExporter()
        exp.exclude_input = True           # Hide code cells
        exp.exclude_input_prompt = True
        body, _ = exp.from_notebook_node(nb)

    cache_file.write_text(body, encoding="utf-8")
    return body

In [3]:
def _list_projects():
    return sorted([p.name for p in ARTIFACTS.glob("*") if p.is_dir()])

def _list_runs(proj: str):
    return sorted([r.name for r in (ARTIFACTS/proj/"runs").glob("*") if r.is_dir()])

dd_proj = W.Dropdown(options=_list_projects(), description="Project", layout=W.Layout(width="300px"))
dd_run  = W.Dropdown(options=_list_runs(dd_proj.value) if dd_proj.value else [], description="Run", layout=W.Layout(width="300px"))
dd_proj.observe(lambda ch: setattr(dd_run, "options", _list_runs(dd_proj.value)), names="value")

dd_stack = W.SelectMultiple(options=["python","java","js"], description="TechStack", layout=W.Layout(width="300px", height="120px"))
dd_stat  = W.Dropdown(options=[("大小分布","size_dist"),("数量","count")], description="统计", layout=W.Layout(width="300px"))

btn_load = W.Button(description="加载结果", button_style="primary")
out = W.Output()

display(W.VBox([
    W.HBox([dd_proj, dd_run]),
    W.HBox([dd_stack, dd_stat, btn_load]),
    out
]))

In [6]:
def render():
    out.clear_output()
    proj, run = dd_proj.value, dd_run.value
    if not (proj and run):
        with out: display(HTML("<b>请选择 Project 与 Run</b>"))
        return

    run_dir = str(ARTIFACTS / proj / "runs" / run)
    stacks = list(dd_stack.value) or ["python"]
    cache_dir = _cache_dir_for(proj, run)

    with out:
        display(HTML(f"<h2>Run: {proj}/{run}</h2>"))

        # First level: tech stack; second level: statistic type
        for s in stacks:
            display(HTML(f"<h3>TechStack: {s}</h3>"))
            html = execute_notebook_to_html_cached(
                "notebooks/parts/techstack_panel.ipynb",
                {"RUN_DIR": run_dir, "TECH_STACK": s, "STAT": dd_stat.value},
                cache_dir
            )
            display(HTML(html))

        # Cross-run trends
        display(HTML("<hr><h3>跨 run 趋势</h3>"))
        html = execute_notebook_to_html_cached(
            "notebooks/parts/trend_panel.ipynb",
            {"PROJECT_ID": proj},
            cache_dir   # You can also use artifacts/<project>/panel/embed as the cache directory
        )
        display(HTML(html))

btn_load.on_click(lambda _: render())